In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
#Import dataframes 

df_player_stats = pd.read_csv('DataFiles/player_stats_per_minute.csv')
df_bio = pd.read_csv('DataFiles/df_bio.csv')
data_index = pd.read_csv('DataFiles/data_index.csv')

In [3]:
columns_to_delete = ['PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK',
                     'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK',
                     'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK',
                     'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK',
                     'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK',
                     'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK',
                     'DD2_RANK', 'TD3_RANK', 'WNBA_FANTASY_PTS_RANK']

# Delete the specified columns
df_player_stats.drop(columns=columns_to_delete, inplace=True, errors='ignore')

df_player_stats.rename(columns={'Season_type': 'Season'}, inplace=True)

In [4]:
# Columns to be dropped in 'df_bio' table:
df_bio.drop(columns=['PLAYER_HEIGHT', 'PLAYER_HEIGHT_INCHES', 'PLAYER_WEIGHT', 'PTS', 'REB', 'AST'], inplace=True, errors='ignore')

In [12]:
df_bio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18282 entries, 0 to 18281
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PLAYER_ID          18282 non-null  int64  
 1   PLAYER_NAME        18281 non-null  object 
 2   TEAM_ID            18281 non-null  float64
 3   TEAM_ABBREVIATION  18281 non-null  object 
 4   AGE                18281 non-null  float64
 5   COLLEGE            15612 non-null  object 
 6   COUNTRY            18281 non-null  object 
 7   DRAFT_YEAR         18282 non-null  object 
 8   DRAFT_ROUND        18173 non-null  object 
 9   DRAFT_NUMBER       18161 non-null  object 
 10  GP                 18282 non-null  int64  
 11  NET_RATING         18282 non-null  float64
 12  OREB_PCT           18282 non-null  float64
 13  DREB_PCT           18282 non-null  float64
 14  USG_PCT            18282 non-null  float64
 15  TS_PCT             18282 non-null  float64
 16  AST_PCT            182

In [6]:
# Merge 'df_player_stats' and ‘df_bio‘ on common keys ‘PLAYER_ID‘ and ‘Year‘ in order to get 'COUNTRY' column.
merged_df_stats_bio = pd.merge(df_player_stats, df_bio, on=['PLAYER_ID', 'Year', 'Season'], how='inner')

columns_to_delete_merged = ['GP_y','NET_RATING','OREB_PCT','DREB_PCT','USG_PCT', 'TS_PCT','AST_PCT']

merged_df_stats_bio.drop(columns=columns_to_delete_merged, inplace=True)          

In [7]:
# Checking the result of 'merged_df_stats_bio'
merged_df_stats_bio.shape

(18282, 42)

In [8]:
# Merging 'merged_df_stat_bio' with player index table on player-id to get player´s role/position.
final_merged_df = pd.merge(merged_df_stats_bio, data_index, left_on='PLAYER_ID', right_on='PERSON_ID', how='inner')

# Dropping all duplicate columns with trailing letter 'y'.
columns_to_drop = [col for col in final_merged_df.columns if 'y' in col]
final_merged_df.drop(columns=columns_to_drop, inplace=True)

# Dropping redundant columns
columns_to_drop_2 = [
    'PLAYER_LAST_NAME',
    'PLAYER_FIRST_NAME',
    'PLAYER_SLUG',
    'TEAM_ID',
    'TEAM_SLUG',
    'IS_DEFUNCT',
    'TEAM_NAME',
    'NICKNAME',
    'TEAM_ABBREVIATION',
    'ROSTER_STATUS',
    'STATS_TIMEFRAME', 
    'PERSON_ID',
    'TEAM_CITY'
]

final_merged_df.drop(columns=columns_to_drop_2, inplace=True)

In [9]:
# Drop '_x' at the end of column names 
for column in final_merged_df.columns: 
    if '_x' in column: 
        column_name = column.replace('_x', '')
        final_merged_df.rename(columns={column:column_name}, inplace=True)

# Change regular season name at 'Season' column  
final_merged_df['Season'] = final_merged_df['Season'].apply(lambda x:'Regular Season' if x =='Regular%20Season' else x)

In [10]:
# Output of final dataframe
nj = final_merged_df[(final_merged_df['PLAYER_NAME'] == 'Nikola Jokic')]
nj

# We don´t need:
# PFD, PLAYER_HEIGHT, PLAYER_HEIGHT_INCHES, PLAYER_WEIGHT, PTS, REB, AST

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,Year,Season,COLLEGE,COUNTRY,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,JERSEY_NUMBER,POSITION,HEIGHT,WEIGHT,FROM_YEAR,TO_YEAR
15457,203999,Nikola Jokic,1610612743,DEN,21.0,80,31,49,0.388,1732.551667,0.18,0.35,0.512,0.02,0.05,0.333,0.09,0.11,0.811,0.10,0.22,0.32,0.11,0.06,0.05,0.03,0.03,0.12,0.11,0.46,2015-16,Regular Season,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15458,203999,Nikola Jokic,1610612743,DEN,22.0,73,37,36,0.507,2037.728333,0.24,0.42,0.578,0.02,0.07,0.324,0.09,0.11,0.825,0.10,0.25,0.35,0.18,0.08,0.03,0.03,0.02,0.11,0.11,0.60,2016-17,Regular Season,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15459,203999,Nikola Jokic,1610612743,DEN,23.0,75,43,32,0.573,2442.505000,0.21,0.41,0.499,0.05,0.11,0.396,0.11,0.13,0.850,0.08,0.25,0.33,0.19,0.09,0.04,0.02,0.02,0.09,0.16,0.57,2017-18,Regular Season,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15460,203999,Nikola Jokic,1610612743,DEN,24.0,80,53,27,0.663,2503.586667,0.25,0.48,0.511,0.03,0.11,0.307,0.12,0.14,0.821,0.09,0.25,0.35,0.23,0.10,0.04,0.02,0.03,0.09,0.17,0.64,2018-19,Regular Season,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15461,203999,Nikola Jokic,1610612743,DEN,24.0,14,7,7,0.500,556.468333,0.24,0.47,0.506,0.04,0.10,0.393,0.12,0.14,0.846,0.10,0.23,0.33,0.21,0.06,0.03,0.02,0.02,0.10,0.14,0.63,2018-19,Playoffs,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15462,203999,Nikola Jokic,1610612743,DEN,25.0,73,46,27,0.630,2335.405000,0.24,0.46,0.528,0.03,0.11,0.314,0.11,0.13,0.817,0.07,0.23,0.30,0.22,0.10,0.04,0.02,0.01,0.10,0.16,0.62,2019-20,Regular Season,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15463,203999,Nikola Jokic,1610612743,DEN,25.0,19,9,10,0.474,694.310000,0.26,0.49,0.519,0.06,0.14,0.429,0.10,0.11,0.835,0.05,0.22,0.27,0.16,0.09,0.03,0.02,0.02,0.10,0.14,0.67,2019-20,Playoffs,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15464,203999,Nikola Jokic,1610612743,DEN,26.0,72,47,25,0.653,2487.781667,0.29,0.52,0.566,0.04,0.10,0.388,0.14,0.16,0.868,0.08,0.23,0.31,0.24,0.09,0.04,0.02,0.02,0.08,0.17,0.76,2020-21,Regular Season,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15465,203999,Nikola Jokic,1610612743,DEN,26.0,10,4,6,0.400,345.413333,0.34,0.66,0.509,0.06,0.15,0.377,0.13,0.16,0.836,0.11,0.23,0.34,0.14,0.07,0.02,0.03,0.01,0.07,0.17,0.86,2020-21,Playoffs,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023
15466,203999,Nikola Jokic,1610612743,DEN,27.0,74,46,28,0.622,2475.591667,0.31,0.53,0.583,0.04,0.12,0.337,0.15,0.19,0.810,0.08,0.33,0.41,0.24,0.11,0.04,0.03,0.03,0.08,0.20,0.81,2021-22,Regular Season,NaN,Serbia,2014,2,41,15,C,6-11,284.0,2015,2023


In [11]:
final_merged_df.to_csv('merged_per_minute.csv', index=False)